# Instance Segmentation with Berlin DOP, 10 cm resolution. The annotations are in COCO format


In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [14]:
!nvidia-RAM


/bin/bash: nvidia-RAM: command not found


In [15]:
!cat /proc/meminfo

MemTotal:       26696420 kB
MemFree:         4799444 kB
MemAvailable:   25084896 kB
Buffers:          145148 kB
Cached:         19883140 kB
SwapCached:            0 kB
Active:          2475016 kB
Inactive:       18678476 kB
Active(anon):     954296 kB
Inactive(anon):      508 kB
Active(file):    1520720 kB
Inactive(file): 18677968 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               468 kB
Writeback:            12 kB
AnonPages:       1125268 kB
Mapped:          1040328 kB
Shmem:              1204 kB
KReclaimable:     504252 kB
Slab:             575456 kB
SReclaimable:     504252 kB
SUnreclaim:        71204 kB
KernelStack:        6832 kB
PageTables:        14348 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    13348208 kB
Committed_AS:    4752984 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       49784 kB
VmallocChunk:          0 kB
Percpu:          

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Jan 29 18:45:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [18]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.8)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#import torch
#assert torch.__version__.startswith("1.8")   # need to manually install torch 1.8 if Colab changes its default version
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
#!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

## Get data

In [19]:
# check CUDA version
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [20]:
import torch
print(torch.__version__, torch.cuda.is_available())

1.10.0+cu111 True


In [21]:
!ls -d /usr/local/cuda-*

/usr/local/cuda-10.0  /usr/local/cuda-11    /usr/local/cuda-11.1
/usr/local/cuda-10.1  /usr/local/cuda-11.0


In [22]:
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [4]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html


In [5]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog

Importing data

In [6]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "/content/drive/MyDrive/Berlin DOP 2020 10cm SidesofRoad/Train_annotations.json", "/content/drive/MyDrive/Berlin DOP 2020 10cm SidesofRoad/Train")
register_coco_instances("my_dataset_valid", {}, "/content/drive/MyDrive/Berlin DOP 2020 10cm SidesofRoad/Valid_annotations.json", "/content/drive/MyDrive/Berlin DOP 2020 10cm SidesofRoad/Valid")
register_coco_instances("my_dataset_test", {}, "", "/content/drive/MyDrive/Berlin DOP 2020 10cm SidesofRoad/Test")

## Register data-set

In order to use a dataset with Detectron2 we need to register it. For more information check out the official documentation.

In [7]:
#visualize training data
my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train")
dataset_dicts = DatasetCatalog.get("my_dataset_train")

import random
from detectron2.utils.visualizer import Visualizer

for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=my_dataset_train_metadata, scale=1)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

Output hidden; open in https://colab.research.google.com to view.

## Train model

Now, let's fine-tune a pretrained FasterRCNN instance segmentation model on the microcontroller data-set.

In [ ]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

In [8]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0025
cfg.SOLVER.MAX_ITER = 700
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()



[01/29 18:46:39 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f10217.pkl: 178MB [00:08, 20.0MB/s]                           
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (3, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (3,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (8, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (8,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to i

[01/29 18:46:54 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[01/29 18:47:11 d2.utils.events]:  eta: 0:08:47  iter: 19  total_loss: 1.955  loss_cls: 0.8788  loss_box_reg: 0.1724  loss_mask: 0.6841  loss_rpn_cls: 0.1325  loss_rpn_loc: 0.0387  time: 0.8021  data_time: 0.5741  lr: 7.0289e-05  max_mem: 1862M
[01/29 18:47:26 d2.utils.events]:  eta: 0:08:32  iter: 39  total_loss: 1.338  loss_cls: 0.3381  loss_box_reg: 0.2016  loss_mask: 0.6334  loss_rpn_cls: 0.07513  loss_rpn_loc: 0.04391  time: 0.7883  data_time: 0.5293  lr: 0.00014165  max_mem: 1927M
[01/29 18:47:43 d2.utils.events]:  eta: 0:08:13  iter: 59  total_loss: 1.178  loss_cls: 0.2509  loss_box_reg: 0.2306  loss_mask: 0.5134  loss_rpn_cls: 0.0614  loss_rpn_loc: 0.06515  time: 0.8081  data_time: 0.5882  lr: 0.000213  max_mem: 1927M
[01/29 18:47:49 d2.utils.events]:  eta: 0:07:48  iter: 79  total_loss: 1.054  loss_cls: 0.1946  loss_box_reg: 0.1913  loss_mask: 0.4885  loss_rpn_cls: 0.06147  loss_rpn_loc: 0.05188  time: 0.6830  data_time: 0.0712  lr: 0.00028436  max_mem: 1927M
[01/29 18:47:55 d

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

## Use model for inference

Now, we can perform inference on our validation set by creating a predictor object.

In [9]:
# Evaluation
test_metadata = MetadataCatalog.get("my_dataset_valid")
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4
cfg.DATASETS.TEST = ("my_dataset_valid", )
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_valid", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "my_dataset_valid")
inference_on_dataset(trainer.model, val_loader, evaluator)

WARNING [01/29 18:50:41 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
WARNING [01/29 18:50:42 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[01/29 18:50:42 d2.data.datasets.coco]: Loaded 15 images in COCO format from /content/drive/MyDrive/Berlin DOP 2020 10cm SidesofRoad/Valid_annotations.json
[01/29 18:50:42 d2.data.build]: Distribution of instances among all 3 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|  Ocluded   | 12           |  Parking   | 46           |  Possible  | 3            |
|            |              |            |              |            |              |
|   total    | 61           |            |              |            |              |
[01/29 18:50:42 d2.data.

OrderedDict([('bbox',
              {'AP': 10.516165542172205,
               'AP-Ocluded': 24.173875868417948,
               'AP-Parking': 7.374620758098673,
               'AP-Possible': 0.0,
               'AP50': 19.828103732097258,
               'AP75': 9.652432882169926,
               'APl': 10.693587896055618,
               'APm': 10.780019487196881,
               'APs': nan}),
             ('segm',
              {'AP': 2.705550553534839,
               'AP-Ocluded': 3.3248877682799334,
               'AP-Parking': 4.791763892324584,
               'AP-Possible': 0.0,
               'AP50': 9.005019671767922,
               'AP75': 1.2795327151762794,
               'APl': 4.66087487869666,
               'APm': 5.46009862184523,
               'APs': nan})])

In [10]:
# Printing the names of the categories
for i, name in enumerate(MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes):
    print(i, name)

0 Ocluded
1 Parking
2 Possible


In [11]:
# from detectron2.utils.visualizer import ColorMode
from skimage.measure import find_contours, approximate_polygon, subdivide_polygon
import glob
import json
import numpy as np
from pycocotools import mask
from skimage import measure
import base64
import cv2

# Saving the predictions in lebelme format

In [18]:

for imageName in glob.glob('/content/drive/MyDrive/Berlin DOP 2020 10cm SidesofRoad/For matrices/*.jpeg'):
  
#for d in dataset_dicts:
  im = cv2.imread(imageName)
  outputs = predictor(im)
  masks = outputs['instances'].pred_masks.cpu().numpy()
  categories = outputs['instances'].pred_classes.cpu().numpy()
  encoded = base64.b64encode(open(imageName, "rb").read())
  number_of_masks = masks.shape[0]
  file_name_1 = imageName.replace("/content/drive/MyDrive/Berlin DOP 2020 10cm SidesofRoad/For matrices/","")
  file_name_2 = file_name_1.replace(".jpeg","")
  file_name_3 = file_name_2 + ".json"
  file_name_4 = str("/content/drive/MyDrive/Berlin DOP 2020 10cm SidesofRoad/New Folder/") + str(file_name_3)
  #print(file_name_1, file_name_2, file_name_3, file_name_4)
  main_annotation = {}
  main_annotation['shapes'] = []


  # Iterating for all masks. Every mask has one contour and a category relevant for that
  for x in range(0, number_of_masks):
    # Selecting relevant mask. Number of masks == number of categories
    ground_truth_binary_mask = masks[x,:,:]
    category = categories[x]
    if category == 0:
      cat = "Ocluded"
    elif category == 1:
      cat = "Parked"
    else:
      cat = "Possible"
    #fortran_ground_truth_binary_mask = np.asfortranarray(ground_truth_binary_mask)
    #contours = measure.approximate_polygon(ground_truth_binary_mask, 0.5)
    contours = measure.find_contours(ground_truth_binary_mask, 0.5)
    try:
      app_con = measure.approximate_polygon(np.array(contours[0]), tolerance=0.8)
    except:
      continue
    annotation = {
          "label" : cat,
          #"points": np.flip(contours[0], axis=1).tolist(),
          "points": np.flip(app_con, axis=1).tolist(),
          "group_id": None,
          "shape_type": "polygon",
          "flags": {}}

    main_annotation['shapes'].append(annotation)

  main_annotation["imagePath"] = file_name_1
  main_annotation["imageData"] = str(encoded)[2:-1]
  main_annotation["imageHeight"] = 512
  main_annotation["imageWidth"] = 512
  #print(main_annotation)
  # print(json.dumps(annotation, indent=4,  separators=(',', ':',)))
  with open(file_name_4, 'w') as file:
    json.dump(main_annotation, file,indent=4,  separators=(',', ':',))

  # v = Visualizer(im[:, :, ::-1],
  #                 metadata=test_metadata, 
  #                 scale=1, 
  #                 #instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
  # )
  # v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  # cv2_imshow(v.get_image()[:, :, ::-1])

  # f = open("/content/drive/MyDrive/Cowc for detectrone2 test/COWC_PKLot_Predictions.txt","a")
  # f.write(str(d["file_name"]) +"\n" + str(outputs["instances"])+"\n")
  # f.close()
  # break
  # break

# Visualising the predictions

In [19]:
from detectron2.utils.visualizer import ColorMode
import glob

for imageName in random.sample(glob.glob('/content/drive/MyDrive/Berlin DOP 2020 10cm SidesofRoad/For matrices/*.jpeg'),15):
  im = cv2.imread(imageName)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
                metadata=test_metadata, 
                scale=1
                 )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])

Output hidden; open in https://colab.research.google.com to view.